In [2]:
import os
import pandas as pd
import pickle
from matplotlib import pyplot as plt
import numpy as np
from ribbn_scripts.processing.get_offset import get_offset
from ribbn_scripts.processing.phase_estimate import multidist_multifreq_phase_estimation
from ribbn_scripts.processing.localization import estimate_dph_df,direct_distance_estimate,estimate_k,unwrap_phases
from ribbn_scripts.ref_functions.spec_functions import read_network_analyzer_file, get_theta, get_amplitude, s2z
import pickle

In [3]:
df_all=None
file_names = sorted([f for f in os.listdir("dataframes") if f[0] != '.'], key=lambda x: int(x.split('.')[0]))
for file_name in file_names:
    exp_num=int(file_name.split(".")[0])
    with open(f'dataframes/{file_name}', "rb") as f:
        print(file_name)
        _df = pickle.load(f)
        # Add new columns to _df for all rows
        _df['Run Exp Num'] = exp_num
        if df_all is None:
            df_all=_df
        else:
            df_all = pd.concat([df_all, _df], ignore_index=True)

df_all

0.df
1.df
2.df
3.df
4.df
5.df
6.df
7.df


,Rx,Tx,MPP Start Time (s),MPP Stop Time (s),Voltages (mV),Frequency (MHz),Run Exp Num,NumMPPs
0,Tag1,Tag2,1.770333e+09,1.770333e+09,"[658.69, 194.74, 195.05, 194.9, 194.59, 194.67...",845,0,1
1,Tag2,Tag1,1.770333e+09,1.770333e+09,"[258.75, 281.8, 282.18, 284.39, 281.8, 281.95,...",845,0,1
2,Tag1,Tag2,1.770333e+09,1.770333e+09,"[193.22, 177.73, 177.73, 178.57, 177.81, 177.4...",855,0,1
3,Tag2,Tag1,1.770333e+09,1.770333e+09,"[286.22, 244.11, 245.71, 246.24, 244.64, 246.1...",855,0,1
4,Tag1,Tag2,1.770333e+09,1.770333e+09,"[175.75, 91.97, 91.29, 91.13, 91.29, 91.29, 91...",865,0,1
...,...,...,...,...,...,...,...,...
171,Tag2,Tag1,1.770334e+09,1.770334e+09,"[323.76, 246.7, 247.62, 247.23, 247.23, 248.07...",925,7,1
172,Tag1,Tag2,1.770334e+09,1.770334e+09,"[1885.82, 1409.36, 1409.29, 1409.9, 1409.06, 1...",935,7,1
173,Tag2,Tag1,1.770334e+09,1.770334e+09,"[200.54, 59.09, 58.25, 58.48, 58.1, 58.18, 58....",935,7,1
174,Tag1,Tag2,1.770334e+09,1.770334e+09,"[1411.57, 1013.62, 1014.08, 1013.47, 1014.69, ...",945,7,1


In [26]:
time_per_phase = 10/1000  # s
num_phases=6
default_MPPs=5
num_mpps=df_all.iloc[0]['NumMPPs']*default_MPPs
sampling_rate=1000
plotting=False
ver_lines = [0] + [ (time_per_phase-time_per_phase*0.01) * sampling_rate * (i + 1) for i in range(num_phases*num_mpps)]

processedDF=pd.DataFrame(columns=["Rx","Tx", "Voltages (mV)", "Phase1","Phase3","Phase4","Phase6","Phase7","Phase8",
                                "Frequency (MHz)", "Run Exp Num", "NumMPPs"])
processedDF_aggregated=pd.DataFrame(columns=["Rx","Tx","phase", "median", "std","freq", "dist", 'delta', "Experiment Number", "Unique Exp Number"])

if plotting:
    plt.figure(figsize=(15,20))

unique_exp_no=0
phase_order=[1,3,4,6,7,8]

for df_idx in range(len(df_all)):

    voltages=df_all.iloc[df_idx]['Voltages (mV)']
    phase_medians={
        1:[],
        3:[],
        4:[],
        6:[],
        7:[],
        8:[],
    }
    for idx,v in enumerate(ver_lines):
        if idx<len(ver_lines)-1:
            phase_medians[phase_order[int(idx%num_phases)]].append(np.median(voltages[int(ver_lines[idx]):int(ver_lines[idx+1])]))   
        if plotting:
            plt.subplot(len(df_all)//3+1, 3, df_idx+1)
            if idx%num_phases==0:
                plt.axvline(x=v, color='b', linestyle='-')
            else:
                plt.axvline(x=v, color='r', linestyle='--')
    
    for phase_out_idx in range(len(phase_medians[1])):
        entry={
            "Rx": df_all.iloc[df_idx]["Rx"],
            "Tx":df_all.iloc[df_idx]["Tx"],
            "Voltages (mV)":df_all.iloc[df_idx]["Voltages (mV)"],
            "Phase1":phase_medians[1][phase_out_idx],
            "Phase3":phase_medians[3][phase_out_idx],
            "Phase4":phase_medians[4][phase_out_idx],
            "Phase6":phase_medians[6][phase_out_idx],
            "Phase7":phase_medians[7][phase_out_idx],
            "Phase8":phase_medians[8][phase_out_idx],
            "Frequency (MHz)":df_all.iloc[df_idx]["Frequency (MHz)"],
            "Run Exp Num":df_all.iloc[df_idx]["Run Exp Num"],
            # "Tape Dist (m)":df_all.iloc[df_idx]["Tape dist"],
            # "Laser Dist (m)":df_all.iloc[df_idx]["Laser dist"],
            "Dist (m)":1.2,
            "NumMPPs":df_all.iloc[df_idx]["NumMPPs"],
        }
        processedDF=pd.concat([processedDF,pd.DataFrame([entry])],ignore_index=True)
        # print("Yo",phase_out_idx)
        # print(phase_medians)
        
    for phase in phase_order:
        entry_aggregated={
            "Rx":df_all.iloc[df_idx]["Rx"],
            "Tx":df_all.iloc[df_idx]["Tx"],
            "phase": str(phase),
            "median": np.mean(phase_medians[phase]), # mean of medians
            "std": np.std(phase_medians[phase]),
            "freq": df_all.iloc[df_idx]["Frequency (MHz)"]*1e6,
            "dist":1.2,
            'delta': max(phase_medians[phase])-min(phase_medians[phase]),
            "Experiment Number":df_all.iloc[df_idx]["Run Exp Num"],
            "Unique Exp Number":unique_exp_no,
            "allVoltages": phase_medians[phase]
        }
        processedDF_aggregated=pd.concat([processedDF_aggregated, pd.DataFrame([entry_aggregated])], ignore_index=True)
    unique_exp_no+=1
    
    # Identify and print bad MPPs
    # bad=False
    # for k in phase_medians.keys():
    #     assert(len(phase_medians[k])==num_mpps)
    #     if np.std(phase_medians[k])>1:
    #         d=1.2
    #         print(f"<== Bad data found at {df_idx} df_idx at phase {k} at T2T dist {d}.", end="\t")
    #         print(f"Num vals: {len(phase_medians[k])}, mean: {np.mean(phase_medians[k])}, stdev: {np.std(phase_medians[k])} ==>")
    
    if plotting:
        plt.plot(voltages,'.')
        plt.title(df_idx)
        plt.ylim([np.percentile(voltages, 1), np.percentile(voltages, 99)])
        plt.tight_layout()
    # break
if plotting:
    plt.show()
    

    

/var/folders/hm/7p8fzz2x2y10pbydpxcs7vw00000gn/T/ipykernel_14492/1137437962.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  processedDF=pd.concat([processedDF,pd.DataFrame([entry])],ignore_index=True)
/var/folders/hm/7p8fzz2x2y10pbydpxcs7vw00000gn/T/ipykernel_14492/1137437962.py:76: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  processedDF_aggregated=pd.concat([processedDF_aggregated, pd.DataFrame([entry_aggregated])], ignore_index=True)


In [27]:
with open("processedDF.pkl", 'wb') as f:
    pickle.dump(processedDF,f)
processedDF

,Rx,Tx,Voltages (mV),Phase1,Phase3,Phase4,Phase6,Phase7,Phase8,Frequency (MHz),Run Exp Num,NumMPPs,Dist (m)
0,Tag1,Tag2,"[658.69, 194.74, 195.05, 194.9, 194.59, 194.67...",194.900,199.820,210.495,184.900,187.340,192.530,845,0,1,1.2
1,Tag1,Tag2,"[658.69, 194.74, 195.05, 194.9, 194.59, 194.67...",195.085,199.435,210.690,185.210,187.800,192.265,845,0,1,1.2
2,Tag1,Tag2,"[658.69, 194.74, 195.05, 194.9, 194.59, 194.67...",195.090,200.045,210.540,184.900,187.535,192.265,845,0,1,1.2
3,Tag1,Tag2,"[658.69, 194.74, 195.05, 194.9, 194.59, 194.67...",195.120,199.705,210.610,185.170,187.305,192.760,845,0,1,1.2
4,Tag1,Tag2,"[658.69, 194.74, 195.05, 194.9, 194.59, 194.67...",195.165,200.770,211.415,185.400,188.070,193.140,845,0,1,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,Tag2,Tag1,"[43.98, 19.19, 16.37, 18.88, 16.21, 19.87, 17....",18.880,23.120,15.945,17.660,21.670,25.065,945,7,1,1.2
876,Tag2,Tag1,"[43.98, 19.19, 16.37, 18.88, 16.21, 19.87, 17....",16.480,20.025,13.655,16.865,20.410,25.105,945,7,1,1.2
877,Tag2,Tag1,"[43.98, 19.19, 16.37, 18.88, 16.21, 19.87, 17....",16.515,20.485,14.230,16.100,21.290,25.520,945,7,1,1.2
878,Tag2,Tag1,"[43.98, 19.19, 16.37, 18.88, 16.21, 19.87, 17....",18.155,21.135,13.690,16.555,20.945,25.295,945,7,1,1.2


In [21]:
processedDF_aggregated

,Rx,Tx,phase,median,std,freq,dist,delta,Experiment Number,Unique Exp Number,allVoltages
0,Tag1,Tag2,1,74.402,0.093199,775000000.0,0.306,0.230,0,0,"[74.5, 74.465, 74.465, 74.31, 74.27]"
1,Tag1,Tag2,3,72.206,0.060778,775000000.0,0.306,0.155,0,0,"[72.25, 72.175, 72.255, 72.25, 72.1]"
2,Tag1,Tag2,4,71.892,0.039064,775000000.0,0.306,0.115,0,0,"[71.905, 71.945, 71.87, 71.91, 71.83]"
3,Tag1,Tag2,6,92.944,0.075789,775000000.0,0.306,0.230,0,0,"[92.93, 92.97, 92.81, 92.97, 93.04]"
4,Tag1,Tag2,7,84.749,0.055893,775000000.0,0.306,0.150,0,0,"[84.765, 84.8, 84.8, 84.65, 84.73]"
...,...,...,...,...,...,...,...,...,...,...,...
1375,Tag2,Tag1,3,16.053,0.245308,995000000.0,1.908,0.650,4,229,"[15.91, 15.83, 16.48, 15.870000000000001, 16.175]"
1376,Tag2,Tag1,4,16.060,0.259210,995000000.0,1.908,0.685,4,229,"[15.754999999999999, 16.215, 15.79, 16.44, 16.1]"
1377,Tag2,Tag1,6,16.030,0.250579,995000000.0,1.908,0.615,4,229,"[15.75, 16.14, 15.715, 16.215, 16.33]"
1378,Tag2,Tag1,7,16.029,0.340755,995000000.0,1.908,0.875,4,229,"[16.555, 15.68, 16.29, 15.715, 15.905000000000..."


In [28]:
# freq_range=np.array(range(775,1000,10))*1e6
# all_freqs, all_freqs_unwrapped, all_freqs_theoretical, all_freqs_theoretical_unwrapped, all_errors, \
#     selected_experiments_all = multidist_multifreq_phase_estimation(freq_range,processedDF_aggregated,correction_factor=0,plot=True)

# with open("no_correction_all_errs.pkl",'wb') as f:
#     pickle.dump(all_errors,f)
    
# with open("no_correction_all_phases.pkl",'wb') as f:
#     pickle.dump(all_freqs,f)

# with open("no_correction_all_phases_unwrapped.pkl",'wb') as f:
#     pickle.dump(all_freqs_unwrapped,f)    

# with open("theoretical_phase.pkl",'wb') as f:
#     pickle.dump(all_freqs_theoretical,f)    

### With offset correction using the ground truth.

In [29]:
# correction_offsets={}
# all_errors=None


# all_freqs, all_freqs_unwrapped, all_freqs_theoretical, all_freqs_theoretical_unwrapped, _,_=\
#     multidist_multifreq_phase_estimation(freq_range,processedDF_aggregated,correction_factor=0,plot=False)


# for freq in freq_range:
#     offset,best_error,errors=get_offset(all_freqs[freq],freq)
#     # print(f"Freq {freq//1e6};\t Mean:{np.mean(errors)};\t Median:{np.median(errors)};\t Std:{np.std(errors)}")
#     # if all_errors is None:
#     #     all_errors=errors
#     # else:
#     #     all_errors=np.concat((all_errors,errors))
#     correction_offsets[freq]=offset



# all_freqs, all_freqs_unwrapped, all_freqs_theoretical, all_freqs_theoretical_unwrapped, all_errors,_=\
#     multidist_multifreq_phase_estimation(freq_range,processedDF_aggregated,correction_factor=correction_offsets,plot=True)


# with open("gt_correction_all_errs.pkl",'wb') as f:
#     pickle.dump(all_errors,f)
    
# with open("gt_correction_all_phases.pkl",'wb') as f:
#     pickle.dump(all_freqs,f)

# with open("gt_correction_all_phases_unwrapped.pkl",'wb') as f:
#     pickle.dump(all_freqs_unwrapped,f)    

### With correction offsets from pretrained

In [30]:
# correction_offsets=pickle.load(open("/Users/manavjeet/git/T2TExperiments/DistExperiments/correction_offsets.pkl",'rb'))

# all_freqs, all_freqs_unwrapped, all_freqs_theoretical, all_freqs_theoretical_unwrapped, all_errors,_=\
#     multidist_multifreq_phase_estimation(freq_range,processedDF_aggregated,correction_factor=correction_offsets,plot=True)


# with open("pretrained_correction_all_errs.pkl",'wb') as f:
#     pickle.dump(all_errors,f)
    
# with open("pretrained_correction_all_phases.pkl",'wb') as f:
#     pickle.dump(all_freqs,f)

# with open("pretrained_correction_all_phases_unwrapped.pkl",'wb') as f:
#     pickle.dump(all_freqs_unwrapped,f)